## Audit Report Parser

This notebook will parse the auditor's reports for the years 2013 to 2021 and extract the audited financial statements (Balance Sheets and Income Statements), from which it will extract values for revenue, expenses and net assets to be used for analysis.  Here are the steps that will be taken:  
- Import all the pdf audit reports from the folder where they were stored
- Combine all of them into one pdf which will be used as input in FastAPI
- Convert the merged pdf file to images
- Convert the images to text using tesseract engine
- Applying regular expression to extract data from the text

### Importing the PDF reports stored in a folder and merging them to one PDF file

In [1]:
!pip install PyPDF2


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from glob import glob
from PyPDF2 import PdfMerger
import pandas as pd

In [2]:
# Defining the folder containing all audit reports of all historical years

folder = glob(r'C:\Users\amari\OneDrive\Documents\Python_Code basics\GIA_financials\backend\resources\*')
folder

['C:\\Users\\amari\\OneDrive\\Documents\\Python_Code basics\\GIA_financials\\backend\\resources\\2013-12-31 Goodwill Industries of Alberta FS-AUD (PBC).pdf',
 'C:\\Users\\amari\\OneDrive\\Documents\\Python_Code basics\\GIA_financials\\backend\\resources\\2014-12-31 Goodwill Industries of Alberta FS-AUD (PBC).pdf',
 'C:\\Users\\amari\\OneDrive\\Documents\\Python_Code basics\\GIA_financials\\backend\\resources\\2015-12-31 Goodwill Industries of Alberta FS-AUD (PBC).pdf',
 'C:\\Users\\amari\\OneDrive\\Documents\\Python_Code basics\\GIA_financials\\backend\\resources\\2016-12-31 Goodwill Industries of Alberta FS-AUD (PBC).pdf',
 'C:\\Users\\amari\\OneDrive\\Documents\\Python_Code basics\\GIA_financials\\backend\\resources\\2017-12-31 Goodwill Industries of Alberta (Registered Society) FS-AUD-F ..Signed.pdf',
 'C:\\Users\\amari\\OneDrive\\Documents\\Python_Code basics\\GIA_financials\\backend\\resources\\2018-12-31 Goodwill Industries of Alberta (Registered Society) FS-AUD-F (PBC) SIGNED.pd

In [5]:
# Number of audit years
len(folder)

9

In [6]:
folder[0]

'C:\\Users\\amari\\OneDrive\\Documents\\Python_Code basics\\GIA_financials\\backend\\resources\\2013-12-31 Goodwill Industries of Alberta FS-AUD (PBC).pdf'

In [3]:
# Merging all the PDF reports to one PDF file called 'merged_reports.pdf'

merger = PdfMerger()
for pdf in folder:
    merger.append(pdf)

outfilename = r"C:\Users\amari\OneDrive\Documents\Python_Code basics\GIA_financials\backend\uploads\merged_reports1.pdf"
outfile = open(outfilename, "wb")
merger.write(outfile)
merger.close()


### Converting the merged PDF to images

In [5]:
from pdf2image import convert_from_path

In [7]:
#
file_path = r"C:\Users\amari\OneDrive\Documents\Python_Code basics\GIA_financials\backend\uploads\merged_reports.pdf"
img = convert_from_path(file_path, poppler_path = r'C:\poppler-22.12.0\Library\bin')
print(img)

[<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23AA118AC10>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23AA118AEE0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23AA1190040>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23AA1190190>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23B13051D90>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23B13051C40>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23B1305C4F0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23B1305CDC0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23B1305CE50>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23B1305C220>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23B130667F0>, <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1700x2200 at 0x23B13066820>, <PIL.PpmImagePl

In [8]:
# Number of images/pages in the merged report
len(img)

155

### Converting the image to text

In [9]:
import pytesseract

In [10]:
pytesseract.pytesseract.tesseract_cmd=r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [11]:
#Getting all Income Statements and Balance sheets

searchstring1 = "Statement of Operations"
searchstring2 = "Interest revenue"
incomestatements = ''
searchstring3 = "Statement of Financial Position"
searchstring4 = "Accounts payable"
balancesheets = ''
fs = ''

for i in img:
    text = pytesseract.image_to_string(i, lang='eng')
    if (searchstring1 in text and searchstring2 in text) or (searchstring3 in text and searchstring4 in text):
        fs = fs + '' + text
print(fs)

GOODWILL INDUSTRIES OF ALBERTA

Statement of Operations

Year ended December 31, 2013, with comparative information for 2012

2013 2012

Revenues
Commercial $ 18,446,374 $ 18,090,637
Provincial contract fees 2,208,589 2,108,213
United Way 177,728 174,991
Donations, grants and contributions 81,848 148,360
Amortization of deferred capital contributions 62,167 55,857
Unrealized gain on investments 12,448 12,736
Interest revenue 4,963 3,503
20,994,117 20,594,297

Expenses
Human resource costs (note 14) 13,291,714 12,992,010
Physical resource costs (note 14) 5,397,023 5,115,306
Other operating costs (note 14) 1,663,814 1,573,217
Amortization of property and equipment 523,050 461,945
Material costs (note 14) 426,964 337,304
21,302,565 20,479,782
(Deficiency) excess of revenues over expenses $ (308,448) $ 114,515

See accompanying notes to financial statements.
GOODWILL INDUSTRIES OF ALBERTA

Statement of Financial Position

December 31, 2013, with comparative information for 2012

2013 2012


### Using Regex to extract data from the text

In [12]:
import re

In [13]:
IncomeStatements = '''
GOODWILL INDUSTRIES OF ALBERTA

Statement of Operations

Year ended December 31, 2013, with comparative information for 2012

2013 2012

Revenues
Commercial $ 18,446,374 $ 18,090,637
Provincial contract fees 2,208,589 2,108,213
United Way 177,728 174,991
Donations, grants and contributions 81,848 148,360
Amortization of deferred capital contributions 62,167 55,857
Unrealized gain on investments 12,448 12,736
Interest revenue 4,963 3,503
20,994,117 20,594,297

Expenses
Human resource costs (note 14) 13,291,714 12,992,010
Physical resource costs (note 14) 5,397,023 5,115,306
Other operating costs (note 14) 1,663,814 1,573,217
Amortization of property and equipment 523,050 461,945
Material costs (note 14) 426,964 337,304
21,302,565 20,479,782
(Deficiency) excess of revenues over expenses $ (308,448) $ 114,515

See accompanying notes to financial statements.


GOODWILL INDUSTRIES OF ALBERTA

Statement of Operations

Year ended December 31, 2014, with comparative information for 2013

2014 2013

Revenues
Commercial $ 19,225,212 $ 18,446,374
Provincial contract fees (note 6) 2,489,045 2,208,589
United Way 156,856 177,728
Donations, grants and contributions 67,483 81,848
Amortization of deferred capital contributions 62,446 62,167
Unrealized gain on investments 10,266 12,448
Interest revenue 284 4,963
22,011,592 20,994,117

Expenses
Human resource costs (note 14) 13,664,328 13,291,714
Occupancy costs (note 14) 5,654,644 5,397,023
Other operating costs (note 14) 1,839,774 1,663,814
Amortization of property and equipment 595,072 523,050
Material costs (note 14) 461,187 426,964
22,215,005 21,302,565
(Deficiency) of revenues over expenses $ (203,413) $ (308,448)

See accompanying notes to financial statements.


GOODWILL INDUSTRIES OF ALBERTA

Statement of Operations

Year ended December 31, 2015, with comparative information for 2014

2015 2014

Revenues
Commercial $ 21,426,431 $ 19,225,212
Provincial contract fees (note 6) 2,421,110 2,489,045
United Way 162,491 156,856
Donations, grants and contributions 58,258 67,483
Amortization of deferred capital contributions 68,907 62,446
Unrealized gain on investments 8,611 10,266
Interest revenue 421 284
24,146,229 22,011,592

Expenses
Human resource costs (note 15) 14,168,964 13,664,328
Occupancy costs (note 15) 5,843,996 5,654,644
Other operating costs (note 15) 1,923,848 1,839,774
Amortization of property and equipment 705,158 595,072
Material costs (note 15) 436,912 461,187
23,078,878 22,215,005
Excess (deficiency) of revenues over expenses $ 1,067,351 $ (203,413)

See accompanying notes to financial statements.


GOODWILL INDUSTRIES OF ALBERTA

Statement of Operations

Year ended December 31, 2016, with comparative information for 2015

2016 2015

Revenues
Commercial $ 21,596,330 $ 21,426,431
Provincial contract fees (note 7) 2,384,903 2,421,110
United Way 164,928 162,491
Donations, grants and contributions 51,473 58,258
Amortization of deferred capital contributions 35,661 68,907
Unrealized gain on investments 6,320 8,611
Interest revenue 5,393 421
24,245,008 24,146,229

Expenses
Human resources (note 16) 14,064,283 14,168,964
Occupancy (note 16) 6,119,256 5,843,996
Other operating (note 16) 1,814,635 1,923,848
Amortization of property and equipment 686,287 705,158
Materials (note 16) 630,978 436,912
23,315,439 23,078,878
Excess of revenues over expenses $ 929,569 $ 1,067,351

See accompanying notes to financial statements.


GOODWILL INDUSTRIES OF ALBERTA (Registered Society)

Statement of Operations

Year ended December 31, 2017, with comparative information for 2016

2017 2016

Revenues
Commercial $ 24,597,943 $ 21,596,330
Provincial contract fees (note 7) 2,446,961 2,384,903
United Way 124,305 164,928
Donations, grants and contributions 134,897 51,473
Amortization of deferred capital contributions 30,964 35,661
Unrealized gain on investments 5,611 6,320
Interest revenue 4,206 5,393
27,344,887 24,245,008

Expenses
Human resources (note 16) 16,571,203 14,064,283
Occupancy (note 16) 7,847,365 6,119,256
Other operating (note 16) 2,024,013 1,814,635
Amortization of property and equipment 1,002,971 686,287
Materials (note 16) 892,638 630,978
28,338,190 23,315,439
Excess (deficiency) of revenues over expenses $ (993,303) $ 929,569

See accompanying notes to financial statements.


GOODWILL INDUSTRIES OF ALBERTA (Registered Society)

Statement of Operations

Year ended December 31, 2018, with comparative information for 2017

2018 2017
Revenues
Commercial $ 26,737,544 $ 24,597,943
Provincial contract fees (note 7) 2,382,561 2,446,961
Donations, grants and contributions 124,487 134,897
Amortization of deferred capital contributions 25,336 30,964
Interest revenue 6,994 4,206
Unrealized gain on investments 3,451 5,611
United Way - 124,305
29,280,373 27,344,887
Expenses
Human resources (note 14) 17,562,393 16,571,203
Occupancy (note 14) 8,173,857 7,847,365
Other operating (note 14) 2,229,033 2,024,013
Amortization of property and equipment 1,166,158 1,002,971
Materials (note 14) 780,436 892,638
29,911,877 28,338,190
(Deficiency) of revenues over expenses $ (631,504) $ (993,303)

oa Senha, Sistine aan i SSS ed

See accompanying notes to financial statements.


GOODWILL INDUSTRIES OF ALBERTA (Registered Society)

Statement of Operations

Year ended December 31, 2019, with comparative information for 2018

2019 2018

Operating revenues

Commercial $ 30,274,056 $ 26,737,544

Provincial contract fees 2,204,619 2,382,561

Donations, grants and contributions 272,570 124,487

Interest revenue 13,367 10,445

32,764,612 29,255,037

Operating expenses (note 13)

Human resources 18,957,729 17,562,393

Occupancy 9,222,993 8,507,149

Other operating 2,226,654 2,229,033

Materials 1,144,885 780,436

31,552,261 29,079,011

Excess of revenues over expenses before the under-noted 1,212,351 176,026
Amortization of deferred capital contributions 25,337 25,336
Amortization of deferred tenant inducements 480,079 333,292
Amortization of property and equipment (1,188,578) (1,166,158)
Lease termination fee (712,619) -
Loss on disposal of property and equipment (83,790) -
Deficiency of revenues over expenses $ (267,220) $ (631,504)

See accompanying notes to financial statements.


GOODWILL INDUSTRIES OF ALBERTA (Registered Society)

Statement of Operations

Year ended December 31, 2020, with comparative information for 2019

2020 2019

Operating revenues

Commercial $ 23,614,342 $ 30,274,056

Federal subsidies (note 13) 2,295,714 -

Provincial contract fees 1,387,065 2,204,619

Donations, grants and contributions 60,396 272,570

Interest revenue 2,128 13,367

27,359,645 32,764,612

Operating expenses (note 12)

Human resources 15,047,405 18,957,729

Occupancy 9,521,057 9,222,993

Other operating 1,665,017 2,226,654

Materials 1,167,831 1,144,885

27,401,310 31,552,261

(Deficiency) excess of revenues over expenses
before the under-noted (41,665) 1,212,351
Amortization of deferred capital contributions 22,500 25,337
Amortization of deferred tenant inducements 458,136 480,079
Amortization of property and equipment (1,089,313) (1,188,578)
Lease termination fee - (712,619)
Loss on disposal of property and equipment - (83,790)
Deficiency of revenues over expenses $ (650,342) $ (267,220)

See accompanying notes to financial statements.


GOODWILL INDUSTRIES OF ALBERTA (Registered Society)

Statement of Operations

Year ended December 31, 2021, with comparative information for 2020

2021 2020
Operating revenues
Commercial $ 33,514,059 $ 23,614,342
Federal subsidies (note 14) 688,637 2,295,714
Provincial contract fees 2,060,788 1,387,065
Donations, grants and contributions 221,419 60,396
Interest revenue 5,147 2,128
36,490,050 27,359,645
Operating expenses (note 13)
Human resources 19,236,285 15,047,405
Occupancy 12,004,258 9,521,057
Other operating 2,249,229 1,665,017
Materials 1,484,921 1,167,831
34,974,693 27,401,310
Excess (deficiency) of revenues over expenses
before the undernoted 1,515,357 (41,665)
Amortization of deferred capital contributions 22,500 22,500
Amortization of tenant inducements 517,564 458,136
Amortization of property and equipment (1,155,994) (1,089,313)
Gain on disposal of property and equipment 41,599 -
Excess (deficiency) of revenues over expenses $ 941,026 $ (650,342)

See accompanying notes to financial statements.

'''

In [14]:
BalanceSheets = '''
GOODWILL INDUSTRIES OF ALBERTA

Statement of Financial Position

December 31, 2013, with comparative information for 2012

2013 2012
ASSETS
Current
Cash and cash equivalents $ 685,805 $ 954,184
Investments (note 3) 317,736 207,199
Accounts receivable 194,258 270,841
Inventory 6,630 7,190
Prepaid expenses and deposits 49,995 56,895
1,254,424 1,496,309
Investments (note 3) 219,524 317,613
Prepaid expenses and deposits 317,811 319,641
Property and equipment (note 4) 2,324,810 2,617,059
$ 4,116,569 $ 4,750,622
LIABILITIES
Current
Accounts payable and accrued liabilities (note 11) $ 1,483,988 $ 1,338,910
Deferred revenues 77,208 67,486
Deferred contributions (note 6) 130,768 326,898
Current portion of obligations under capital lease (note 8) 20,244 19,397
1,712,208 1,752,691
Deferred capital contributions (note 7) 122,447 184,614
Obligations under capital lease (note 8) 38,725 58,961
Accrued benefit liability (note 10) - 24,000
Straight-line rent and tenant inducements payable (note 9) 1,504,531 1,683,250
3,377,911 3,703,516
Net assets 738,658 1,047,106

$ 4,116,569 $ 4,750,622
Commitments (note 12)
See accompanying notes to financial statements.

On behalf of the Board:

, Director , Director



GOODWILL INDUSTRIES OF ALBERTA

Statement of Financial Position

December 31, 2014, with comparative information for 2013

2014 2013
ASSETS
Current
Cash and cash equivalents $ 537,968 $ 685,805
Investments (note 3) 320,756 317,736
Accounts receivable 208,843 194,258
Inventory 1,785 6,630
Prepaid expenses and deposits 32,625 49,995
1,101,977 1,254,424
Investments (note 3) 226,770 219,524
Prepaid expenses and deposits 333,297 317,811
Property and equipment (note 4) 2,338,481 2,324,810
$ 4,000,525 $ 4,116,569
LIABILITIES
Current
Accounts payable and accrued liabilities (note 11) $ 1,918,956 $ 1,483,988
Deferred revenues 57,902 77,208
Deferred contributions (note 6) 57,271 130,768
Current portion of obligations under capital lease (note 8) 17,800 20,244
2,051,929 1,712,208
Deferred capital contributions (note 7) 128,702 122,447
Obligations under capital lease (note 8) 21,122 38,725
Straight-line rent and tenant inducements payable (note 9) 1,263,527 1,504,531
3,465,280 3,377,911
Net assets 535,245 738,658

$ 4,000,525 $ 4,116,569

Commitments (note 12)
See accompanying notes to financial statements.

On behalf of the Board:

, Director , Director



GOODWILL INDUSTRIES OF ALBERTA

Statement of Financial Position

December 31, 2015, with comparative information for 2014

2015 2014
ASSETS
Current
Cash $ 1,392,919 $ 537,968
Investments (note 3) 332,792 320,756
Accounts receivable 412,864 208,843
Prepaid expenses, deposits and other 41,351 34,410
2,179,926 1,101,977
Investments (note 3) 223,345 226,770
Prepaid expenses and deposits 340,619 333,297
Property and equipment (note 4) 2,320,701 2,338,481
$ 5,064,591 $ 4,000,525
LIABILITIES
Current
Accounts payable and accrued liabilities (note 11) $ 1,841,350 $ 1,918,956
Deferred revenues 56,860 57,902
Deferred contributions (note 6) 412,414 57,271
Current portion of obligations under capital lease (note 8) 19,235 17,800
2,329,859 2,051,929
Deferred capital contributions (note 7) 59,795 128,702
Obligations under capital lease (note 8) 1,887 21,122
Straight-line rent and tenant inducements payable (note 9) 1,070,454 1,263,527
3,461,995 3,465,280
Net assets 1,602,596 535,245

$ 5,064,591 $ 4,000,525

Commitments (note 12)
See accompanying notes to financial statements.

On behalf of the Board:

, Director , Director



GOODWILL INDUSTRIES OF ALBERTA

Statement of Financial Position

December 31, 2016, with comparative information for 2015

2016 2015
ASSETS
Current
Cash $ 1,815,125 $ 1,392,919
Investments (note 3) 450,777 332,792
Accounts receivable 682,354 412,864
Prepaid expenses, deposits and other 212,712 41,351
3,160,968 2,179,926
Investments (note 3) 386,680 223,345
Prepaid expenses and deposits 392,444 340,619
Property and equipment (note 4) 3,789,429 2,320,701
$ 7,729,521 $ 5,064,591
LIABILITIES
Current
Accounts payable and accrued liabilities (note 6) $ 2,409,959 $ 1,841,350
Deferred revenues 50,140 56,860
Deferred contributions (note 7) 415,989 412,414
Current portion of obligation under capital lease (note 9) 1,887 19,235
Current portion of long-term debt (note 10) 68,994 -
2,946,969 2,329,859
Deferred capital contributions (note 8) 24,134 59,795
Obligations under capital lease (note 9) - 1,887
Long-term debt (note 10) 331,006 -
Straight-line rent and tenant inducements (note 11) 1,895,247 1,070,454
5,197,356 3,461,995
Net assets 2,532,165 1,602,596
$ 7,729,521 $ 5,064,591
Credit facility (note 5)
Commitments (note 12)
See accompanying notes to financial statements.
On behalf of the Board:
, Director , Director



—S Z = , Director

GOODWILL INDUSTRIES OF ALBERTA (Registered Society)

Statement of Financial Position

December 31, 2017, with comparative information for 2016

2017 2016
ASSETS
Current
Cash and cash equivalents $ 1,046,845 $ 1,815,125
Investments (note 3) 568,068 450,777
Accounts receivable 1,059,476 682,354
Prepaid expenses, deposits and other 207,838 212,712
2,882,227 3,160,968
Investments (note 3) 275,000 386,680
Prepaid expenses and deposits 544,225 392,444
Property and equipment (note 4) 6,678,586 3,789,429
$ 10,380,038 $ 7,729,521
LIABILITIES
Current
Accounts payable and accrued liabilities (note 6) $ 2,705,188 $ 2,409,959
Deferred revenues 62,922 50,140
Deferred contributions (note 7) 430,907 415,989
Current portion of obligation under capital lease - 1,887
Current portion of long-term debt (note 9) 151,192 68,994
3,350,209 2,946,969
Deferred capital contributions (note 8) 193,170 24,134
Long-term debt (note 9) 1,679,814 331,006
Straight-line rent and tenant inducements (note 10) 3,617,983 1,895,247
8,841,176 5,197,356
Net assets 1,538,862 2,532,165

$ 10,380,038 $ 7,729,521

Credit facility (note 5)
Commitments and contingencies (note 11)

See accompanying notes to financial statements.

On behalf of the Board:

, Director



GOODWILL INDUSTRIES OF ALBERTA (Registered Society)

Statement of Financial Position

December 31, 2018, with comparative information for 2017

2018 2017
ASSETS
Current
Cash and cash equivalents $ 985,833 $ 1,046,845
Investments (note 3) 500,000 568,068
Accounts receivable 309,903 1,059,476
Prepaid expenses, deposits and other 200,845 207,838
1,996,581 2,882,227
Investments (note 3) 275,000 275,000
Prepaid expenses and deposits 617,867 544,225
Property and equipment (note 4) 6,261,708 6,678,586
$ 9,151,156 $ 10,380,038
LIABILITIES
Current
Accounts payable and accrued liabilities (note 6) $ 1,681,116 $ 2,705,188
Deferred revenues 80,335 62,922
Deferred contributions (note 7) 590,490 430,907
Current portion of long-term debt (note 9) 444,180 151,192
2,796,121 3,350,209
Deferred capital contributions (note 8) 167,834 193,170
Long-term debt (note 9) 1,235,634 1,679,814
Straight-line rent and tenant inducements (note 10) 4,044,209 3,617,983
8,243,798 8,841,176
Net assets 907,358 4,538,862

$ 9,151,156 $ 10,380,038

Credit facility (note 5)
Commitments and contingencies (note 11)

See accompanying notes to financial statements.

On behalf of the Board:

Le lj YA, , Director Dwitig >= Director


GOODWILL INDUSTRIES OF ALBERTA (Registered Society)

Statement of Financial Position

December 31, 2019, with comparative information for 2018

2019 2018

ASSETS

Current

Cash and cash equivalents $ 1,828,340 $ 985,833
Investments (note 3) 512,013 500,000
Accounts receivable 513,579 309,903
Prepaid expenses and deposits 709,339 818,712
3,563,271 2,614,448
Non-current investments (note 3) 275,000 275,000
Property and equipment (note 4) 6,366,669 6,261,708

$ 10,204,940 $ 9,151,156

LIABILITIES AND NET ASSETS

Current
Accounts payable and accruals (note 6) $ 1,662,463 $ 1,681,116
Deferred revenues 678,628 670,825
Scheduled cash repayments for long-term debt (note 7) 472,301 444,180
2,813,392 2,796,121
Callable debt (note 7) 350,000 -
3,163,392 2,796,121
Long-term debt (note 7) 763,334 1,235,634
Deferred capital contributions (note 8) 142,497 167,834
Straight-line rent and tenant inducements (note 9) 5,495,579 4,044,209
9,564,802 8,243,798
Net assets 640,138 907,358

$ 10,204,940 $ 9,151,156
Credit facility (note 5)
Commitments and contingencies (note 10)
Subsequent event (note 14)

See accompanying notes to financial statements.

On behalf of the Board:

, Director , Director



GOODWILL INDUSTRIES OF ALBERTA (Registered Society)

Statement of Financial Position

December 31, 2020, with comparative information for 2019

2020 2019
ASSETS
Current
Cash and cash equivalents $ 2,566,410 $ 1,828,340
Investments (note 3) 500,000 512,013
Accounts receivable 651,105 513,579
Prepaid expenses and deposits 893,063 709,339
4,610,578 3,563,271
Non-current investments (note 3) 275,000 275,000
Property and equipment (note 4) 5,740,027 6,366,669
$ 10,625,605 $ 10,204,940
LIABILITIES AND NET ASSETS
Current
Accounts payable and accruals (note 5) $ 3,366,617 $ 1,662,463
Deferred revenues 660,252 678,628
Current portion of long-term debt (note 6) 473,555 472,301
4,500,424 2,813,392
Callable debt (note 6) 342,923 350,000
4,843,347 3,163,392
Long-term debt (note 6) 561,377 763,334
Unamortized deferred capital contributions (note 7) 119,997 142,497
Straight-line rent and tenant inducements (note 8) 5,111,088 5,495,579
10,635,809 9,564,802
Net (deficiency) assets (10,204) 640,138
$ 10,625,605 $ 10,204,940
Credit facility (note 6)
Commitments and contingencies (note 10)
Impacts of COVID-19 (note 13)
See accompanying notes to financial statements.
On behalf of the Board:
, Director , Director



GOODWILL INDUSTRIES OF ALBERTA (Registered Society)

Statement of Financial Position

December 31, 2021, with comparative information for 2020

2021 2020

ASSETS

Current

Cash and cash equivalents $ 2,401,680 $ 2,566,410
Investments (note 3) 775,000 500,000
Accounts receivable 310,013 651,105
Prepaid expenses and deposits 841,595 893,063
4,328,288 4,610,578
Non-current investments (note 3) - 275,000
Property and equipment (note 4) 7,384,662 5,740,027

$ 11,712,950 $ 10,625,605

LIABILITIES AND NET ASSETS (DEFICIENCY)

Current
Accounts payable and accruals (note 5) $ 1,976,105 $ 3,366,617
Deferred revenues 694,599 660,252
Current portion of long-term debt (note 6) 765,650 816,478
Current portion of obligation under capital lease (note 7) 15,388 -
3,451,742 4,843,347
Long-term debt (note 6) 851,282 561,377
Obligations under capital lease (note 7) 63,705 -
Unamortized deferred capital contributions (note 8) 97,497 119,997
Straight-line rent and tenant inducements (note 9) 6,317,902 5,111,088
10,782,128 10,635,809
Net assets (deficiency) 930,822 (10,204)

$ 11,712,950 $ 10,625,605
Credit facility (note 6)
Commitments and contingencies (note 11)
Impacts of COVID-19 (note 14)

See accompanying notes to financial statements.

On behalf of the Board:

, Direct 7 , Director

'''

In [15]:
# Getting the net assets

pattern = "Net assets \(deficiency\) ([\d,]+)|Net assets ([\d,]+)|Net \(deficiency\) assets ([\(\d,\)]+)"
netAssets =  re.findall(pattern, BalanceSheets)  #returns tuples including empty values for each year
netAssets = [(tuple(int(x) if x.isdigit() else x for x in _ if x))[0] for _ in netAssets]
netAssets = [s.replace('(','-') for s in netAssets]
netAssets = [s.replace(')','') for s in netAssets]
netAssets = [s.replace(',','') for s in netAssets]
print(netAssets)
print(type(netAssets[0]))
    

['738658', '535245', '1602596', '2532165', '1538862', '907358', '640138', '-10204', '930822']
<class 'str'>


In [16]:
# Getting the revenues

pattern = "([\d,]+) [\d,]+\n\nExpenses|([\d,]+) [\d,]+\nExpenses|([\d,]+) [\d,]+\n\nOperating expenses|([\d,]+) [\d,]+\nOperating expenses"  # grab the 2 values above 'Expense' line and use the first. NB:text has \n\n before 'Expense'
revenue =  re.findall(pattern, IncomeStatements, flags=re.DOTALL)  #returns tuples including empty values for each of the conditionns above not met
revenue = [(tuple(int(x) if x.isdigit() else x for x in _ if x))[0] for _ in revenue] # getting only the digits
revenue = [s.replace('(','-') for s in revenue]
revenue = [s.replace(')','') for s in revenue]
revenue = [s.replace(',','') for s in revenue]
print(revenue)

['20994117', '22011592', '24146229', '24245008', '27344887', '29280373', '32764612', '27359645', '36490050']


In [17]:
# Getting the Expenses

pattern = "(\([\d,]+\)) [\d,]+\n\(Deficiency\)|([\d,]+) [\d,]+\n\(Deficiency\)|([\d,]+) \([\d,]+\)\n\(Deficiency\)|([\d,]+) [\d,]+\nExcess \(deficiency\)|([\d,]+) [\d,]+\nExcess of|([\d,]+) [\d,]+\n\nExcess of| (-) \([\d,]+\)\nDeficiency| (-) - \nDeficiency"  # grab the 2 values above 'Deficiencey' or 'Excess of..' lines and use the first. NB:text may have \n\n before 'See'; some values are negatives with brackets
expenses =  re.findall(pattern, IncomeStatements, flags=re.DOTALL)  #returns tuples including empty values for each of the conditionns above not met
expenses = [(tuple(int(x) if x.isdigit() else x for x in _ if x))[0] for _ in expenses] # getting only the digits
expenses = [s.replace('(','-') for s in expenses]
expenses = [s.replace(')','') for s in expenses]
expenses = [s.replace(',','') for s in expenses]
print(expenses)

['21302565', '22215005', '23078878', '23315439', '28338190', '29911877', '31552261', '-', '34974693']


In [18]:
# Getting the Net Surplus/Deficit

pattern = "revenues over expenses \$ (\([\d,]+\)) \$ \([\d,]+\)|revenues over expenses \$ ([\d,]+) \$ \([\d,]+\)|revenues over expenses \$ ([\d,]+) \$ [\d,]+|revenues over expenses \$ (\([\d,]+\)) \$ [\d,]+"  # grab the 2 values after 'See accomp...' line and use the first. NB:text may have \n\n before 'revenue over expenses'; some values are negatives with brackets
netSurplus =  re.findall(pattern, IncomeStatements)  #returns tuples including empty values for each of the conditionns above not met
netSurplus = [(tuple(int(x) if x.isdigit() else x for x in _ if x))[0] for _ in netSurplus] # getting only the digits
netSurplus = [s.replace('(','-') for s in netSurplus]
netSurplus = [s.replace(')','') for s in netSurplus]
netSurplus = [s.replace(',','') for s in netSurplus]
print(netSurplus)

['-308448', '-203413', '1067351', '929569', '-993303', '-631504', '-267220', '-650342', '941026']
